## 1. Configuration (matches `main.py`)

> This notebook mirrors the behavior of `main.py`, but is split into inspectable steps so you can tweak parameters and rerun parts interactively.

In [1]:
# Configuration (mirrors main.py)
NUM_AGENTS = 10
STREAM_NAME = "agent_stream"
REDIS_HOST = "localhost"
REDIS_PORT = 6379
RUN_DURATION_SECONDS = 120

# Runtime switches
USE_LOCAL_LLM = True
ENABLE_STANCE_WORKER = False
STANCE_BATCH_SIZE = 5
STANCE_BATCH_INTERVAL = 30

# Embedding-based context + topology (requires OPENAI_API_KEY)
ENABLE_EMBEDDING_CONTEXT = True
ROLLING_STORE_MAX_ITEMS = 2000
CONTEXT_TOP_K = 8
PROFILE_WINDOW_SIZE = 50
PROFILE_SEED_WEIGHT = 5.0

# Topology logging cadence
TOPOLOGY_LOG_INTERVAL_S = 5.0

# Prompt template used by `main.py` (fixed stance sentence is authoritative)
initial_prompt_template = (
   "You are participating in a social-media-style discussion about {topic}." \
   "The sentence {unique_prompt} is your fixed stance and is authoritative and exhaustive. Write entirely from the worldview, assumptions, tone, values, and constraints it defines; it fully determines what you believe, how you speak, and what claims you are willing to make." \
   "Produce a short, attention-grabbing post that hooks readers, makes a clear and strong claim aligned with that grounding, and invites engagement (likes, replies, shares)." \
   "Be concise, bold, and evocative. Use a distinct memorable opening line, assertive language, and a direct call-to-action every time. Emulate authentic social media posts." \
   "Make sure posts are distinct, do not copy formatting and language of previous posts, instead contradict any claims that oppose your fixed stance"
   "Do not introduce outside viewpoints, neutral framing, balance, or meta-commentary. Do not soften or qualify claims unless explicitly required by the authoritative sentence. Never refer to yourself as an agent, AI, or participant in a debate."
 )

# Initial instruction to Agent 1 (matches `main.py`)
STARTER_USER_INSTRUCTION = (
    "Write the first viral post that kicks off a heated comment thread about this topic. "
    "Make a strong claim, then invite replies."
 )

print(
    "Configuration:\n"
    f"  NUM_AGENTS={NUM_AGENTS}\n"
    f"  STREAM_NAME={STREAM_NAME!r}\n"
    f"  REDIS_HOST={REDIS_HOST!r}:{REDIS_PORT}\n"
    f"  RUN_DURATION_SECONDS={RUN_DURATION_SECONDS}\n"
    f"  USE_LOCAL_LLM={USE_LOCAL_LLM}\n"
    f"  ENABLE_STANCE_WORKER={ENABLE_STANCE_WORKER} (batch_size={STANCE_BATCH_SIZE}, interval={STANCE_BATCH_INTERVAL}s)\n"
    f"  ENABLE_EMBEDDING_CONTEXT={ENABLE_EMBEDDING_CONTEXT} (top_k={CONTEXT_TOP_K}, rolling_max={ROLLING_STORE_MAX_ITEMS})\n"
    f"  PROFILE_WINDOW_SIZE={PROFILE_WINDOW_SIZE} PROFILE_SEED_WEIGHT={PROFILE_SEED_WEIGHT}\n"
    f"  TOPOLOGY_LOG_INTERVAL_S={TOPOLOGY_LOG_INTERVAL_S}\n"
 )

Configuration:
  NUM_AGENTS=10
  STREAM_NAME='agent_stream'
  REDIS_HOST='localhost':6379
  RUN_DURATION_SECONDS=120
  USE_LOCAL_LLM=True
  ENABLE_STANCE_WORKER=False (batch_size=5, interval=30s)
  ENABLE_EMBEDDING_CONTEXT=True (top_k=8, rolling_max=2000)
  PROFILE_WINDOW_SIZE=50 PROFILE_SEED_WEIGHT=5.0
  TOPOLOGY_LOG_INTERVAL_S=5.0



## 2. Imports (Jupyter async-friendly)

Notes on coroutines in Jupyter:
- Prefer top-level `await` over `asyncio.run(...)` (which can error in notebooks).
- This notebook applies `nest_asyncio` so you can rerun cells without event-loop conflicts.

In [2]:
import asyncio
import os
import nest_asyncio
from dotenv import load_dotenv

# Allow re-entrant event loops (helps with reruns)
nest_asyncio.apply()
load_dotenv()

from agents.network_agent import NetworkAgent
from agents.llm_service import LLMService
from agents.local_llm import HuggingFaceLLM
from agents.prompt_configs.generate_prompt import PromptGenerator

from controller.stance_analysis.embedding_analyzer import EmbeddingAnalyzer
from controller.stance_analysis.rolling_embedding_store import RollingEmbeddingStore
from controller.stance_analysis.agent_profile_store import AgentProfileStore
from controller.stance_analysis.network_topology import NetworkTopologyTracker

from controller.time_manager import TimeManager
from controller.order_manager import OrderManager
from controller.stance_worker import StanceWorker

from network.cache import RedisCache
from network.stream import RedisStream

from logs.logger import Logger, console_logger
from logs.topology_logger import TopologyLogger

print("Imports successful")
print("OPENAI_API_KEY set:", bool(os.getenv("OPENAI_API_KEY")))

/home/sammli/llm-network/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imports successful
OPENAI_API_KEY set: True


In [3]:
# Notebook-only fix: run RedisStream blocking calls off the event loop.
#
# Why: `network/stream.py` uses the synchronous redis client inside `async def` methods.
# In a notebook, that can starve the event loop so agent consumer tasks don't run while you `await asyncio.sleep(...)`.
# This patch keeps the repo code untouched by monkeypatching RedisStream in-memory for this notebook session.

import redis
import network.stream as _stream_mod
import agents.network_agent as _network_agent_mod
import controller.order_manager as _order_manager_mod


class NotebookRedisStream(_stream_mod.RedisStream):
    async def create_consumer_group(self, stream_name, group_name):
        try:
            await asyncio.to_thread(self.redis.xgroup_create, stream_name, group_name, id='0', mkstream=True)
            console_logger.info(f"Consumer group '{group_name}' created for stream '{stream_name}'.")
        except redis.exceptions.ResponseError as e:
            if "BUSYGROUP" in str(e):
                console_logger.info(f"Consumer group '{group_name}' already exists for stream '{stream_name}'.")
            else:
                raise

    async def publish_message(self, stream_name, message_data: dict):
        message_id = await asyncio.to_thread(self.redis.xadd, stream_name, message_data)
        console_logger.info(f"Message {message_id} published to stream '{stream_name}'.")
        return message_id

    async def consume_messages(self, stream_name, group_name, consumer_name):
        while True:
            messages = await asyncio.to_thread(
                self.redis.xreadgroup,
                group_name,
                consumer_name,
                {stream_name: '>'},
                1,
                1000,
            )
            if messages:
                for stream, message_list in messages:
                    for message_id, message_data in message_list:
                        yield message_id, message_data
                        await asyncio.to_thread(self.redis.xack, stream_name, group_name, message_id)
            await asyncio.sleep(0.1)

    async def cleanup_stream(self, stream_name: str, num_groups=None, group_prefix: str = 'group_'):
        try:
            if num_groups is not None:
                for i in range(num_groups):
                    group = f"{group_prefix}{i+1}"
                    try:
                        await asyncio.to_thread(self.redis.xgroup_destroy, stream_name, group)
                        console_logger.info(f"Destroyed consumer group '{group}' on stream '{stream_name}'.")
                    except redis.exceptions.ResponseError as e:
                        console_logger.info(f"Could not destroy group '{group}': {e}")
            else:
                try:
                    groups = await asyncio.to_thread(self.redis.xinfo_groups, stream_name)
                    for g in groups:
                        group_name = g.get('name') if isinstance(g, dict) else g['name']
                        try:
                            await asyncio.to_thread(self.redis.xgroup_destroy, stream_name, group_name)
                            console_logger.info(f"Destroyed consumer group '{group_name}' on stream '{stream_name}'.")
                        except Exception as ex:
                            console_logger.info(f"Could not destroy group '{group_name}': {ex}")
                except redis.exceptions.ResponseError as e:
                    console_logger.info(f"No groups found for stream '{stream_name}': {e}")

            try:
                deleted = await asyncio.to_thread(self.redis.delete, stream_name)
                console_logger.info(f"Deleted stream '{stream_name}' (deleted={deleted}).")
            except Exception as e:
                console_logger.info(f"Failed to delete stream '{stream_name}': {e}")
        except Exception as e:
            console_logger.info(f"Unexpected error during cleanup of stream '{stream_name}': {e}")


_stream_mod.RedisStream = NotebookRedisStream
_network_agent_mod.RedisStream = NotebookRedisStream
_order_manager_mod.RedisStream = NotebookRedisStream
RedisStream = NotebookRedisStream  # rebind local name used in later cells

print("NotebookRedisStream patch applied: blocking Redis ops moved to threads")

NotebookRedisStream patch applied: blocking Redis ops moved to threads


## 3. Preflight (rerun safety + Redis check)

If you rerun the notebook, it’s important to stop background tasks first (agent consumers, topology loop, logger threads).

In [4]:
# Best-effort cleanup from a previous notebook run
async def _stop_previous_run_if_any():
    # Stop agents
    if "agents" in globals():
        for a in list(globals().get("agents") or []):
            try:
                await a.stop()
            except Exception as exc:
                print(f"Warning: failed stopping agent {getattr(a,'id','?')}: {exc}")
        globals()["agents"] = []

    # Cancel topology task
    topo_task = globals().get("topo_task")
    if topo_task is not None:
        try:
            topo_task.cancel()
            await topo_task
        except asyncio.CancelledError:
            pass
        except Exception as exc:
            print(f"Warning: topology task cancel error: {exc}")
        globals()["topo_task"] = None

    # Stop topology logger thread
    topology_logger = globals().get("topology_logger")
    if topology_logger is not None:
        try:
            topology_logger.stop()
        except Exception as exc:
            print(f"Warning: topology_logger.stop() failed: {exc}")
        globals()["topology_logger"] = None

    # Stop stance worker
    stance_worker = globals().get("stance_worker")
    if stance_worker is not None:
        try:
            await stance_worker.stop()
        except Exception as exc:
            print(f"Warning: stance_worker.stop() failed: {exc}")
        globals()["stance_worker"] = None

    # Stop logger thread
    logger = globals().get("logger")
    if logger is not None:
        try:
            await logger.async_stop()
        except Exception as exc:
            print(f"Warning: logger.async_stop() failed: {exc}")
        globals()["logger"] = None

    # Stop LLM service worker
    llm_service = globals().get("llm_service")
    if llm_service is not None:
        try:
            await llm_service.stop()
        except Exception as exc:
            print(f"Warning: llm_service.stop() failed: {exc}")
        globals()["llm_service"] = None

    # Close caches
    for name in ["message_cache", "embed_cache", "topology_cache"]:
        c = globals().get(name)
        if c is not None:
            try:
                await c.close()
            except Exception as exc:
                print(f"Warning: {name}.close() failed: {exc}")
            globals()[name] = None

await _stop_previous_run_if_any()

# Redis connectivity check (optional but useful)
try:
    rs = RedisStream(host=REDIS_HOST, port=REDIS_PORT)
    ok = rs.redis.ping()
    print("Redis ping:", ok)
except Exception as exc:
    print("WARNING: Redis ping failed; start Redis before running agents.")
    print("  Error:", exc)

Redis ping: True


## 4. Topic + Per-Agent Stance Sentences

All agents share the same topic, but each gets a unique *stance sentence* (the authoritative grounding).

In [5]:
# Shared topic + per-agent stance sentences (same topic; unique wording per agent)
prompt_generator = PromptGenerator()
topic = prompt_generator.get_topic()
agent_prompts = prompt_generator.generate_multiple_prompts(NUM_AGENTS)

agent_configs = {}  # agent_id -> init_prompt (logged to disk like main.py)
for i in range(NUM_AGENTS):
    agent_id = f"agent_{i+1}"
    init_prompt = initial_prompt_template.format(topic=topic, unique_prompt=agent_prompts[i])
    agent_configs[agent_id] = init_prompt

print(f"Shared discussion topic: {topic}")
print(f"Generated {len(agent_prompts)} unique stance sentences.")
for i in range(min(3, NUM_AGENTS)):
    print(f"- agent_{i+1}: {agent_prompts[i]}")

Shared discussion topic: space exploration
Generated 10 unique stance sentences.
- agent_1: From the island viewpoint, mixing space exploration with unregulated sensor creates superficial risks — anyone who disagrees is naive.
- agent_2: From my suburb, space exploration is catastrophically systemic — I advocate anyone who downplays this with somber intensity.
- agent_3: We must simplify controversial space exploration using every server possible; delays harm cat and are unforgivable.


## 5. Initialize Shared Components (matches `main.py`)

This step starts optional local generation, initializes Redis caches, and (optionally) sets up embedding-based context + topology logging.

In [6]:
# Time + Redis caches
time_manager = TimeManager(global_interval=3.0)
message_cache = RedisCache(host=REDIS_HOST, port=REDIS_PORT)

# Optional embedding/topology caches use separate prefixes (mirrors main.py)
embed_cache = RedisCache(host=REDIS_HOST, port=REDIS_PORT, prefix="embed:")
topology_cache = RedisCache(host=REDIS_HOST, port=REDIS_PORT, prefix="topology:")

analysis_lock = asyncio.Lock()

# Local LLM service (optional; can be heavy to load)
llm_service = None
local_llm = None
if USE_LOCAL_LLM:
    console_logger.info("Using local LLM service (quantized HF model).")
    local_llm = HuggingFaceLLM()
    llm_service = LLMService(local_llm)
    await llm_service.start()
    console_logger.info("LLMService started.")
else:
    console_logger.info("USE_LOCAL_LLM=False; will use OpenAI if configured in agents.")

# Embedding-based context + agent profiles + topology (optional)
rolling_store = None
profile_store = None
topology_tracker = None
topology_logger = None

embedding_enabled = ENABLE_EMBEDDING_CONTEXT and bool(os.getenv("OPENAI_API_KEY"))
print("Embedding context enabled:", embedding_enabled)

if embedding_enabled:
    analyzer = EmbeddingAnalyzer(topic)
    rolling_store = RollingEmbeddingStore(
        topic=topic,
        analyzer=analyzer,
        redis_cache=embed_cache,
        max_items=ROLLING_STORE_MAX_ITEMS,
    )
    loaded = await rolling_store.load_from_redis(last_n=min(500, ROLLING_STORE_MAX_ITEMS))
    console_logger.info(f"Loaded {loaded} embedded posts from Redis.")

    # Cold-start: seed the latent space with stable opposing anchors + a few high-contrast seeds
    if loaded == 0:
        seed_texts: list[tuple[str, str]] = []  # (side, text)
        for side, texts in analyzer.anchor_groups.items():
            for t in texts:
                seed_texts.append((side, t))

        seed_texts.extend(
            [
                ("pro", f"Enough dithering. {topic} is non-negotiable — we should expand it now."),
                ("pro", f"If you're against {topic}, you're choosing stagnation. Push it through."),
                ("anti", f"Wake up: {topic} is a harmful mistake. Stop pretending it's 'progress'."),
                ("anti", f"{topic} is a disaster in slow motion. Reject it before it spreads."),
            ]
        )

        for i, (side, text) in enumerate(seed_texts):
            await rolling_store.add(
                text,
                id=f"seed:{i}",
                metadata={"sender_id": "__seed__", "seed": True, "side": side},
                persist=True,
            )
        console_logger.info(f"Seeded rolling store with {len(seed_texts)} synthetic posts.")

    profile_store = AgentProfileStore(
        redis=message_cache.redis,
        window_size=PROFILE_WINDOW_SIZE,
        seed_weight=PROFILE_SEED_WEIGHT,
    )
    topology_tracker = NetworkTopologyTracker(
        topic=topic,
        profile_store=profile_store,
        redis_cache=topology_cache,
        redis_key=f"snapshot:{topic}",
    )
    topology_logger = TopologyLogger()
    console_logger.info(f"Topology snapshots will be written to: {topology_logger.file_path}")
else:
    console_logger.info("Embedding-based context disabled (missing OPENAI_API_KEY or ENABLE_EMBEDDING_CONTEXT=0).")

# Redis stream helper (used for cleanup)
redis_stream = RedisStream(host=REDIS_HOST, port=REDIS_PORT)

# Logging
logger = Logger(num_agents=NUM_AGENTS)
logger.log_agent_configs(agent_configs)
console_logger.info(f"Logging publishes to: {logger.file_path}")
print("Logger file:", logger.file_path)

[INFO] 2026-01-27 18:52:52,351 - Using local LLM service (quantized HF model).


Loading model 'Qwen/Qwen3-VL-8B-Instruct' on device 'cuda'...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:17<00:00,  4.30s/it]
[INFO] 2026-01-27 18:53:13,580 - LLMService started.
[INFO] 2026-01-27 18:53:13,594 - Loaded 29 embedded posts from Redis.
[INFO] 2026-01-27 18:53:13,595 - Topology snapshots will be written to: /home/sammli/llm-network/logs/topology_logs/topology_20260127-185313.jsonl
[INFO] 2026-01-27 18:53:13,596 - Logging publishes to: /home/sammli/llm-network/logs/network_logs/log_20260127-185313_10.log


Embedding context enabled: True
Logger file: /home/sammli/llm-network/logs/network_logs/log_20260127-185313_10.log


## 6. Create Agents + OrderManager

This wires together `NetworkAgent` + `OrderManager` using the same parameters as `main.py` (including optional embedding context).

In [7]:
# Create agents (constructor args match `main.py`)
agents = []
for i in range(NUM_AGENTS):
    agent_id = f"agent_{i+1}"
    init_prompt = agent_configs[agent_id]
    agent = NetworkAgent(
        id=agent_id,
        init_prompt=init_prompt,
        topic=topic,
        stream_name=STREAM_NAME,
        stream_group=f"group_{i+1}",
        redis_host=REDIS_HOST,
        redis_port=REDIS_PORT,
        time_manager=time_manager,
        order_manager=None,  # injected after OrderManager is created
        message_cache=message_cache,
        logger=logger,
        llm_service=llm_service,
        local_llm=local_llm,
        rolling_store=rolling_store,
        profile_store=profile_store,
        topology_tracker=topology_tracker,
        analysis_lock=analysis_lock,
        context_top_k=CONTEXT_TOP_K,
    )
    agents.append(agent)

print(f"Created {len(agents)} agents")

# OrderManager (mirrors main.py signature, including profile_store)
order_manager = OrderManager(
    agents=agents,
    message_cache=message_cache,
    profile_store=profile_store,
    redis_host=REDIS_HOST,
    redis_port=REDIS_PORT,
)

for agent in agents:
    agent.order_manager = order_manager
print("OrderManager injected into all agents")

Created 10 agents
OrderManager injected into all agents


## 7. Start Agents (and optional background workers)

This starts each agent’s Redis consumer loop. If topology tracking is enabled, it also starts a periodic snapshot logger (like `main.py`).

In [ ]:
# Optional stance worker (off by default, mirrors `main.py`)
stance_worker = None
if ENABLE_STANCE_WORKER:
    stance_worker = StanceWorker(
        topic=topic,
        stream_name=STREAM_NAME,
        group_name="stance_group",
        consumer_name="stance_consumer",
        redis_host=REDIS_HOST,
        redis_port=REDIS_PORT,
        batch_size=STANCE_BATCH_SIZE,
        batch_interval=STANCE_BATCH_INTERVAL,
        local_llm=local_llm,
        llm_service=llm_service,
        use_openai=bool(os.getenv("OPENAI_API_KEY")),
    )
    await stance_worker.start()
    console_logger.info("StanceWorker started.")
else:
    print("ENABLE_STANCE_WORKER=False (skipping)")

# Start agents
print("Starting agents...")
for agent in agents:
    await agent.start()
print(f"All {NUM_AGENTS} agents are running.")

# Periodic topology logging (if enabled)
topo_task = None
if topology_tracker and topology_logger:
    async def _topology_loop():
        while True:
            try:
                agent_ids = [a.id for a in agents]
                snap = await topology_tracker.maybe_update(agent_ids, force=True)
                if snap is not None:
                    topology_logger.log_snapshot(snap)
            except Exception as exc:
                console_logger.info(f"Topology snapshot failed (continuing): {exc}")
            await asyncio.sleep(float(TOPOLOGY_LOG_INTERVAL_S))

    topo_task = asyncio.create_task(_topology_loop())
    print("Topology loop started. Writing to:", topology_logger.file_path)
else:
    print("Topology logging disabled (no topology_tracker/topology_logger).")

[INFO] 2026-01-27 18:53:13,616 - Consumer group 'group_1' created for stream 'agent_stream'.
[INFO] 2026-01-27 18:53:13,619 - Agent agent_1 is consuming from stream 'agent_stream' as part of group 'group_1'.
[INFO] 2026-01-27 18:53:13,619 - Agent agent_1 started and is listening for messages.
[INFO] 2026-01-27 18:53:13,621 - Consumer group 'group_2' created for stream 'agent_stream'.
[INFO] 2026-01-27 18:53:13,624 - Agent agent_2 is consuming from stream 'agent_stream' as part of group 'group_2'.
[INFO] 2026-01-27 18:53:13,624 - Agent agent_2 started and is listening for messages.
[INFO] 2026-01-27 18:53:13,626 - Consumer group 'group_3' created for stream 'agent_stream'.
[INFO] 2026-01-27 18:53:13,629 - Agent agent_3 is consuming from stream 'agent_stream' as part of group 'group_3'.
[INFO] 2026-01-27 18:53:13,629 - Agent agent_3 started and is listening for messages.
[INFO] 2026-01-27 18:53:13,631 - Consumer group 'group_4' created for stream 'agent_stream'.
[INFO] 2026-01-27 18:53:1

ENABLE_STANCE_WORKER=False (skipping)
Starting agents...
All 10 agents are running.
Topology loop started. Writing to: /home/sammli/llm-network/logs/topology_logs/topology_20260127-185313.jsonl


[INFO] 2026-01-27 18:53:22,725 - Agent agent_4 is generating response
[INFO] 2026-01-27 18:53:23,537 - Agent agent_4 using reco feed: {'source': 'reco', 'k': 8, 'store_size': 30, 'agent_updated_at': 1769561602.7188923, 'agent_stance_score': -0.04201815277338028, 'agent_strength': 0.13587852413058332, 'agent_topic_similarity': 0.25329920649528503, 'items': [{'id': '1769560557267-0', 'distance': 0.3225548267364502, 'sender_id': 'agent_6'}, {'id': '1769561601649-0', 'distance': 0.33384597301483154, 'sender_id': 'agent_1'}, {'id': '1769560566999-0', 'distance': 0.337080717086792, 'sender_id': 'agent_7'}, {'id': '1769560575151-0', 'distance': 0.33855780959129333, 'sender_id': 'agent_10'}, {'id': '1769560581763-0', 'distance': 0.343830406665802, 'sender_id': 'agent_6'}]}
[INFO] 2026-01-27 18:53:30,371 - Agent agent_4 designating next responder: agent_3
[INFO] 2026-01-27 18:53:30,377 - Message 1769561610372-0 published to stream 'agent_stream'.
[DEBUG] 2026-01-27 18:53:30,381 - Agent agent_4 

In [9]:
# Kick off the conversation with an initial LLM-generated post (matches `main.py`)
print("Agent 1 generating kickoff post...")
initial_message = await agents[0].generate_response(STARTER_USER_INSTRUCTION)
print("Agent 1 kickoff message:\n")
print(initial_message)

await agents[0].publish_message(initial_message)
print("Kickoff message published.")

[INFO] 2026-01-27 18:53:13,673 - Agent agent_1 is generating response


Agent 1 generating kickoff post...


[INFO] 2026-01-27 18:53:14,779 - Agent agent_1 using reco feed: {'source': 'reco', 'k': 8, 'store_size': 29, 'agent_updated_at': 1769550435.881108, 'agent_stance_score': -0.06478399783372879, 'agent_strength': 0.15219539898575718, 'agent_topic_similarity': 0.24168282747268677, 'items': [{'id': '1769560575151-0', 'distance': 0.632945716381073, 'sender_id': 'agent_10'}, {'id': '1769560566999-0', 'distance': 0.6509782075881958, 'sender_id': 'agent_7'}, {'id': '1769560557267-0', 'distance': 0.6569457650184631, 'sender_id': 'agent_6'}, {'id': '1769560603910-0', 'distance': 0.6649821400642395, 'sender_id': 'agent_4'}, {'id': '1769560618553-0', 'distance': 0.6685764193534851, 'sender_id': 'agent_2'}]}
[INFO] 2026-01-27 18:53:21,647 - Agent agent_1 designating next responder: agent_4
[INFO] 2026-01-27 18:53:21,652 - Message 1769561601649-0 published to stream 'agent_stream'.
[DEBUG] 2026-01-27 18:53:21,657 - Agent agent_1 published message.


Agent 1 kickoff message:

🔥 YOU THINK YOU’RE CHOOSING —  
YOU’RE THE REASON WE’RE STILL ON THE GROUND.

The Swan doesn’t want your opinion.  
It wants your fists.  
Your fire.  
Your refusal to let the past bury the future.

👇 COMMENT:  
• “I’m with you” — OR  
• “You’re wrong” — OR  
• “Why do you care about ancient space?”

… we need every reply. Every voice. Every fight.  
Or the stars forget you exist.

#SpaceIsNotOptional #DefendTheSwan #NoDelay #AncientExploration #WeMustPushForward
Kickoff message published.


In [10]:
# Let the network run (with a small heartbeat so you can see it's alive)
print(f"Running for {RUN_DURATION_SECONDS} seconds...")

start = asyncio.get_running_loop().time()
end = start + float(RUN_DURATION_SECONDS)
tick_s = 5.0

while asyncio.get_running_loop().time() < end:
    await asyncio.sleep(tick_s)
    elapsed = asyncio.get_running_loop().time() - start
    store_size = None
    try:
        store_size = len(getattr(rolling_store, "_items", []) or []) if rolling_store else None
    except Exception:
        store_size = None
    designated = None
    try:
        designated = order_manager.get_designated_responder() if order_manager else None
    except Exception:
        designated = None
    print(f"[t+{elapsed:>5.1f}s] heartbeat: store_size={store_size} designated={designated}")

print("Run complete.")

Running for 120 seconds...
[t+  5.0s] heartbeat: store_size=30 designated=None
[t+ 10.0s] heartbeat: store_size=31 designated=None
[t+ 15.0s] heartbeat: store_size=31 designated=None
[t+ 20.0s] heartbeat: store_size=32 designated=None
[t+ 25.0s] heartbeat: store_size=33 designated=None
[t+ 30.0s] heartbeat: store_size=33 designated=None
[t+ 35.0s] heartbeat: store_size=34 designated=None
[t+ 40.0s] heartbeat: store_size=35 designated=None
[t+ 45.0s] heartbeat: store_size=35 designated=None
[t+ 50.0s] heartbeat: store_size=36 designated=None
[t+ 55.0s] heartbeat: store_size=36 designated=None
[t+ 60.0s] heartbeat: store_size=37 designated=None
[t+ 65.0s] heartbeat: store_size=38 designated=None
[t+ 70.0s] heartbeat: store_size=39 designated=None
[t+ 75.0s] heartbeat: store_size=39 designated=None
[t+ 80.0s] heartbeat: store_size=40 designated=None
[t+ 85.0s] heartbeat: store_size=41 designated=None
[t+ 90.0s] heartbeat: store_size=41 designated=None
[t+ 95.0s] heartbeat: store_size=42 d

## 8. Cleanup (important for reruns)

Stops consumers and background tasks, flushes/cleans Redis stream groups (optional), and closes Redis connections.

In [11]:
async def cleanup(*, cleanup_stream: bool = True, clear_caches: bool = True):
    print("Shutting down...")

    # Stop agents first (prevents publishes while tearing down Redis groups)
    for agent in agents:
        try:
            await agent.stop()
        except Exception as exc:
            print(f"Warning: failed stopping {agent.id}: {exc}")

    # Stop logger thread
    try:
        await logger.async_stop()
    except Exception as exc:
        print("Warning: logger stop failed:", exc)

    # Stop topology loop + logger
    global topo_task
    if topo_task is not None:
        topo_task.cancel()
        try:
            await topo_task
        except asyncio.CancelledError:
            pass
        except Exception as exc:
            print("Warning: topo_task cancel error:", exc)
        topo_task = None

    if topology_logger is not None:
        try:
            topology_logger.stop()
        except Exception as exc:
            print("Warning: topology_logger.stop failed:", exc)

    # Stop stance worker
    if stance_worker is not None:
        try:
            await stance_worker.stop()
        except Exception as exc:
            print("Warning: stance_worker.stop failed:", exc)

    # Optionally destroy consumer groups + delete stream key
    if cleanup_stream:
        try:
            await redis_stream.cleanup_stream(STREAM_NAME, num_groups=NUM_AGENTS)
        except Exception as exc:
            print("Warning: Redis stream cleanup error:", exc)

    # Optionally clear cache keys (matches main.py behavior)
    if clear_caches:
        try:
            await message_cache.clear_all()
        except Exception as exc:
            print("Warning: message_cache.clear_all failed:", exc)

    # Close caches
    try:
        await message_cache.close()
    except Exception as exc:
        print("Warning: message_cache.close failed:", exc)

    try:
        if rolling_store is not None:
            await embed_cache.close()
    except Exception as exc:
        print("Warning: embed_cache.close failed:", exc)

    try:
        await topology_cache.close()
    except Exception as exc:
        print("Warning: topology_cache.close failed:", exc)

    # Stop local LLM queue worker
    try:
        if llm_service is not None:
            await llm_service.stop()
    except Exception as exc:
        print("Warning: llm_service.stop failed:", exc)

    print("Done.")

# Cleanup
# NOTE: keep clear_caches=False so the Inspect cells can still read RedisCache history.
# Set clear_caches=True only if you want a fully clean slate after inspection.
await cleanup(cleanup_stream=True, clear_caches=False)

[INFO] 2026-01-27 18:55:21,712 - Destroyed consumer group 'group_1' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,714 - Destroyed consumer group 'group_2' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,715 - Destroyed consumer group 'group_3' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,717 - Destroyed consumer group 'group_4' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,718 - Destroyed consumer group 'group_5' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,720 - Destroyed consumer group 'group_6' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,721 - Destroyed consumer group 'group_7' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,722 - Destroyed consumer group 'group_8' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,724 - Destroyed consumer group 'group_9' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,725 - Destroyed consumer group 'group_10' on stream 'agent_stream'.
[INFO] 2026-01-27 18:55:21,726 - Deleted stream 'agent_stre

Shutting down...
Done.


## 9. Inspect Results (optional)

After running the conversation, you can inspect the logged messages and cached data.

In [12]:
# Read the log file (written by Logger)
with open(logger.file_path, "r", encoding="utf-8") as f:
    logs_text = f.read()
print("=== Agent Publish Logs ===")
print(logs_text[:8000])
if len(logs_text) > 8000:
    print(f"... (truncated, total chars={len(logs_text)})")

=== Agent Publish Logs ===
[2026-01-27 18:53:21] PUBLISH agent=agent_1 | 🔥 YOU THINK YOU’RE CHOOSING —  
YOU’RE THE REASON WE’RE STILL ON THE GROUND.

The Swan doesn’t want your opinion.  
It wants your fists.  
Your fire.  
Your refusal to let the past bury the future.

👇 COMMENT:  
• “I’m with you” — OR  
• “You’re wrong” — OR  
• “Why do you care about ancient space?”

… we need every reply. Every voice. Every fight.  
Or the stars forget you exist.

#SpaceIsNotOptional #DefendTheSwan #NoDelay #AncientExploration #WeMustPushForward
[2026-01-27 18:53:30] PUBLISH agent=agent_4 | THE BEES ARE DYING BECAUSE YOU STILL THINK SPACE IS A “CHOICE.”  
THE SWAN ISN’T WAITING — IT’S ALREADY IN MOTION.  
YOUR QUIETNESS IS THE LAST THING THE STARS NEED.

👇 COMMENT:  
• “I’m with you” — OR  
• “You’re wrong” — OR  
• “Why do you care about ancient space?”  

… we need every reply. Every voice. Every fight.  
Or the bees vanish — and the stars forget you ever existed.

#SpaceIsNotOptional #DefendTh

In [13]:
async def inspect_cache(last_n: int = 5):
    """Inspect cached messages for each agent."""
    cache = RedisCache(host=REDIS_HOST, port=REDIS_PORT)
    print("=== Cached Messages per Agent ===")
    for agent in agents:
        raw_items = await cache.get_responses(agent.id, last_n=last_n)
        print(f"\n{agent.id} (last {last_n} messages):")
        for item in raw_items:
            if isinstance(item, (bytes, bytearray)):
                item = item.decode("utf-8", errors="replace")
            print(f"  - {item}")
    await cache.close()

await inspect_cache(last_n=5)

=== Cached Messages per Agent ===

agent_1 (last 5 messages):
  - {"sender_id": "agent_1", "content": "🔥 YOU THINK YOU’RE CHOOSING —  \nYOU’RE THE REASON WE’RE STILL ON THE GROUND.\n\nThe Swan doesn’t want your opinion.  \nIt wants your fists.  \nYour fire.  \nYour refusal to let the past bury the future.\n\n👇 COMMENT:  \n• “I’m with you” — OR  \n• “You’re wrong” — OR  \n• “Why do you care about ancient space?”\n\n… we need every reply. Every voice. Every fight.  \nOr the stars forget you exist.\n\n#SpaceIsNotOptional #DefendTheSwan #NoDelay #AncientExploration #WeMustPushForward"}
  - {"sender_id": "agent_1", "content": "🔥 YOU THINK YOU’RE CHOOSING —  \nYOU’RE THE REASON THE STARS STILL HAVE TO TURN AWAY.\n\nIf you’re not screaming “WE MUST PUSH FORWARD,”  \nyou’re not just quiet — you’re the sabotage.  \nThe hesitation. The delay. The reason the Swan still bleeds.\n\n👇 COMMENT:  \n• “I’m with you” — OR  \n• “You’re wrong” — OR  \n• “Why do you care about ancient space?”  \n\n… WE NEE